<a href="https://colab.research.google.com/github/unaizanouman/zsPOP/blob/main/zePOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update -y
!apt-get install -y mpich


Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,486 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Pack

In [2]:
%%writefile zepop.cpp

// zepop.cpp
// Simplified educational implementation of ZePoP (MPI + C++).
// Compile: mpicxx -std=c++11 -O2 zepop.cpp -o zepop
// Run: mpirun -np <N> ./zepop
// Example: mpirun -np 8 ./zepop

#include <mpi.h>
#include <bits/stdc++.h>
using namespace std;
const double INF = 1e18;
int main(int argc,char**argv){
    MPI_Init(&argc,&argv);
    int rank,size;
    MPI_Comm_rank(MPI_COMM_WORLD,&rank);
    MPI_Comm_size(MPI_COMM_WORLD,&size);
    int n=size;

    // --- Build a deterministic overlay graph (same at all nodes) ---
    // Each node has neighbors to (i+1)%n and (i-1+n)%n (ring),
    // plus a few extra edges for connectivity.
    std::mt19937_64 rng(12345); // fixed seed -> reproducible
    vector<vector<int>> adj(n);
    vector<unordered_map<int,double>> link_delay(n); // symmetric delays

    for(int i=0;i<n;i++){
        int a=(i+1)%n, b=(i-1+n)%n;
        adj[i].push_back(a); adj[i].push_back(b);
    }
    // Add some extra random edges (deterministic with seed)
    for(int i=0;i<n;i++){
        rng.seed(1000+i); // deterministic per node
        int extra = (i*7+13) % n;
        if(extra!=i && find(adj[i].begin(),adj[i].end(),extra)==adj[i].end()){
            adj[i].push_back(extra);
            adj[extra].push_back(i);
        }
    }
    // Make adjacency unique and symmetric
    for(int i=0;i<n;i++){
        sort(adj[i].begin(),adj[i].end());
        adj[i].erase(unique(adj[i].begin(),adj[i].end()),adj[i].end());
    }
    // Link delays: deterministic pseudo-random between 1.0 and 10.0
    for(int i=0;i<n;i++){
        for(int j:adj[i]){
            if(link_delay[i].count(j)==0){
                rng.seed(5000 + min(i,j)*10000 + max(i,j));
                double d = 1.0 + (rng()%9000)/1000.0; // [1.0, 10.0)
                link_delay[i][j] = d;
                link_delay[j][i] = d;
            }
        }
    }

    // Each process will compute its own distance-vector (dist to all nodes)
    vector<double> dist(n, INF);
    dist[rank] = 0.0;
    // Direct neighbor distances known
    for(auto &p: link_delay[rank]) dist[p.first] = p.second;

    // --- Distributed distance-vector relaxation until convergence ---
    // We'll exchange full distance vectors with neighbors.
    bool global_changed = true;
    int rounds = 0;
    while(true){
        rounds++;
        // Prepare send buffer (dist vector)
        vector<double> sendbuf = dist;
        // Send to each neighbor and receive from each neighbor (pairwise)
        bool local_changed = false;
        for(int nb: adj[rank]){
            // send dist to nb, receive neighbor's dist
            MPI_Send(sendbuf.data(), n, MPI_DOUBLE, nb, 0, MPI_COMM_WORLD);
        }
        // receive from all neighbors and relax
        for(int nb: adj[rank]){
            vector<double> recv(n);
            MPI_Recv(recv.data(), n, MPI_DOUBLE, nb, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            double w = link_delay[rank][nb];
            for(int k=0;k<n;k++){
                if(recv[k] >= INF/2) continue;
                double cand = w + recv[k];
                if(cand + 1e-9 < dist[k]){
                    dist[k] = cand;
                    local_changed = true;
                }
            }
        }
        // global reduction to check convergence
        int lchg = local_changed?1:0;
        int gchg = 0;
        MPI_Allreduce(&lchg, &gchg, 1, MPI_INT, MPI_SUM, MPI_COMM_WORLD);
        if(gchg==0) break;
        if(rounds > n+5) break; // safety
    }

    // Now gather all nodes' dist vectors to form full matrix D[u][v]
    vector<double> all_vec(n * n);
    MPI_Allgather(dist.data(), n, MPI_DOUBLE, all_vec.data(), n, MPI_DOUBLE, MPI_COMM_WORLD);
    // all_vec is concatenation: dist_of_node0 | dist_of_node1 | ...
    // D[u][v] = all_vec[u*n + v] (distance from u to v)
    auto D = [&](int u,int v)->double{
        return all_vec[u*n + v];
    };

    // Compute total delay T_x = sum_{s != x} D[s][x]  (distance FROM others TO x)
    // For undirected symmetric graphs D[s][x]==D[x][s], use that:
    double Tx = 0.0;
    for(int s=0;s<n;s++){
        if(s==rank) continue;
        double d = D(rank, s); // distance from rank to s
        if(d >= INF/2) d = 1e9;
        Tx += d;
    }
    // closeness centrality:
    double closeness = (n>1) ? ( (double)(n-1) / Tx ) : 0.0;

    // Using full matrix, compute next-hop (from x to destination s) via neighbor
    // For each destination s!=x, find neighbor y s.t. link_delay[x][y] + D[y][s] == D[x][s]
    vector<int> next_hop(n, -1);
    for(int s=0;s<n;s++){
        if(s==rank) { next_hop[s] = -1; continue; }
        double best = D(rank, s);
        int choice = -1;
        for(int y: adj[rank]){
            double w = link_delay[rank][y];
            double dy_s = D(y, s);
            if(dy_s >= INF/2) continue;
            if ( fabs( (w + dy_s) - best ) < 1e-6 ){
                if(choice==-1 || y < choice) choice = y;
            } else if ( (w + dy_s) + 1e-9 < best ){
                // numerical slack: sometimes DV may give equalities; accept smaller
                choice = y;
            }
        }
        next_hop[s] = choice; // could be -1 if unreachable
    }

    // Compute Ixy and Oxy for each neighbor (counts)
    unordered_map<int,int> Ixy, Oxy;
    for(int y: adj[rank]){ Ixy[y]=0; Oxy[y]=0; }
    for(int s=0;s<n;s++){
        if(s==rank) continue;
        int nh = next_hop[s];
        if(nh!=-1) Ixy[nh]++; // nodes reached from rank via nh
    }
    // Oxy is simply I_yx (we'll compute via matrix gather: since each node computed I for itself,
    // we can gather everyone's I arrays so each node knows Oxy = I_yx.)
    // Prepare local I array size n (0 except neighbors)
    vector<int> localI(n,0);
    for(auto &kv: Ixy) localI[kv.first] = kv.second;
    vector<int> allI(n*n,0);
    MPI_Allgather(localI.data(), n, MPI_INT, allI.data(), n, MPI_INT, MPI_COMM_WORLD);
    // Now allI[u*n + v] = I_u(v)
    for(int y: adj[rank]){
        Oxy[y] = allI[y*n + rank]; // number of destinations for which y uses rank as next-hop when going from y
    }

    // Determine phi_xy for each neighbor using a combination:
    // prefer delta = Oxy - Ixy > 0  => x better than y
    // else compare Tx vs Ty
    unordered_map<int,bool> phi;
    for(int y: adj[rank]){
        int ixy = Ixy[y];
        int oxy = Oxy[y];
        int delta = oxy - ixy;
        double Ty = 0.0;
        // compute Ty from matrix: sum of distances from node to others
        for(int s=0;s<n;s++){
            if(s==y) continue;
            double d = D(y, s);
            if(d >= INF/2) d = 1e9;
            Ty += d;
        }
        bool okay=false;
        if(delta > 0) okay = true;
        else if(delta < 0) okay = false;
        else{
            if( fabs(Tx - Ty) > 1e-6 ) okay = (Tx < Ty);
            else okay = (rank < y); // tie-break: lower id wins
        }
        phi[y] = okay;
    }

    // Node is a candidate if phi[y]==true for all neighbors y
    bool candidacy = true;
    for(int y: adj[rank]) if(!phi[y]) { candidacy=false; break; }

    // Broadcast candidacy and closeness: gather all nodes' closeness if candidate else -1
    double myC = candidacy ? closeness : -1.0;
    vector<double> allC(n);
    MPI_Allgather(&myC, 1, MPI_DOUBLE, allC.data(), 1, MPI_DOUBLE, MPI_COMM_WORLD);

    // Elect leader: the candidate with max closeness, tie-breaker: lower id
    double bestC = -1.0;
    int leader = -1;
    for(int u=0;u<n;u++){
        if(allC[u] > bestC + 1e-12){
            bestC = allC[u];
            leader = u;
        } else if( fabs(allC[u]-bestC) < 1e-12 && allC[u] > -0.5 ){
            // tie: lower id
            if(u < leader) leader = u;
        }
    }
    // If no candidate found (very unlikely), choose node with max closeness from global computation:
    if(leader == -1){
        // compute global closeness estimates using total delays (lower Tx better)
        vector<double> allTx(n);
        double myTx = Tx;
        MPI_Allgather(&myTx,1,MPI_DOUBLE, allTx.data(),1,MPI_DOUBLE,MPI_COMM_WORLD);
        double bestVal = 1e300; int bestId=0;
        for(int u=0;u<n;u++){
            if(allTx[u] < bestVal){ bestVal = allTx[u]; bestId = u; }
        }
        leader = bestId;
    }

    // Build DCDT: each node picks parent = neighbor that is on shortest path to leader
    int parent = -1;
    if(rank == leader) parent = -1;
    else{
        double best = D(rank, leader);
        int choice = -1;
        for(int y: adj[rank]){
            double w = link_delay[rank][y];
            double dy_l = D(y, leader);
            if(dy_l >= INF/2) continue;
            if( fabs( w + dy_l - best ) < 1e-6 ){
                if(choice==-1 || y < choice) choice = y;
            }
        }
        parent = choice; // could be -1 if unreachable (isolated)
    }

    // Gather parents to root for display
    vector<int> all_parents(n);
    MPI_Allgather(&parent,1,MPI_INT, all_parents.data(),1,MPI_INT, MPI_COMM_WORLD);

    // Print results (only rank 0 prints a neat summary)
    if(rank==0){
        cout<<"=== ZePoP simplified MPI implementation results ===\n";
        cout<<"Nodes: "<<n<<"\n";
        cout<<"Elected leader: "<<leader<<"\n\n";
        cout<<"Parent of each node (DCDT):\n";
        for(int i=0;i<n;i++){
            cout<<" node "<<setw(2)<<i<<" -> parent = ";
            if(all_parents[i]==-1) cout<<"(root / none)";
            else cout<<all_parents[i];
            cout<<"\n";
        }
        cout<<"\nDetailed per-node closeness values (candidates have closeness, others -1):\n";
        for(int i=0;i<n;i++){
            cout<<" node "<<setw(2)<<i<<" closeness = ";
            if(allC[i] < -0.5) cout<<setw(7)<<"-";
            else cout<<fixed<<setprecision(6)<<allC[i];
            cout<<"\n";
        }
        cout<<"\nNote: This simplified implementation computes distributed shortest paths\n"
              "via repeated distance-vector exchange, then gathers full distance matrix\n"
              "so each node can compute next-hops, Ixy/Oxy and decide candidacy.\n";
    }

    MPI_Finalize();
    return 0;
}


Writing zepop.cpp


In [3]:
!mpicxx -std=c++11 -O2 zepop.cpp -o zepop


In [4]:
!mpirun -np 8 ./zepop


--------------------------------------------------------------------------
mpirun has detected an attempt to run as root.

Running as root is *strongly* discouraged as any mistake (e.g., in
defining TMPDIR) or bug can result in catastrophic damage to the OS
file system, leaving your system in an unusable state.

We strongly suggest that you run mpirun as a non-root user.

You can override this protection by adding the --allow-run-as-root option
to the cmd line or by setting two environment variables in the following way:
the variable OMPI_ALLOW_RUN_AS_ROOT=1 to indicate the desire to override this
protection, and OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 to confirm the choice and
add one more layer of certainty that you want to do so.
We reiterate our advice against doing so - please proceed at your own risk.
--------------------------------------------------------------------------


In [7]:
# Install MPI
apt-get update -y
apt-get install -y mpich

# Add your code
cat << 'EOF' > zepop.cpp

#include <mpi.h>
#include <bits/stdc++.h>
using namespace std;
const double INF = 1e18;

int main(int argc,char**argv){
    MPI_Init(&argc,&argv);
    int rank,size;
    MPI_Comm_rank(MPI_COMM_WORLD,&rank);
    MPI_Comm_size(MPI_COMM_WORLD,&size);
    int n=size;

    std::mt19937_64 rng(12345);
    vector<vector<int>> adj(n);
    vector<unordered_map<int,double>> link_delay(n);

    for(int i=0;i<n;i++){
        int a=(i+1)%n, b=(i-1+n)%n;
        adj[i].push_back(a); adj[i].push_back(b);
    }

    for(int i=0;i<n;i++){
        rng.seed(1000+i);
        int extra = (i*7+13) % n;
        if(extra!=i && find(adj[i].begin(),adj[i].end(),extra)==adj[i].end()){
            adj[i].push_back(extra);
            adj[extra].push_back(i);
        }
    }

    for(int i=0;i<n;i++){
        sort(adj[i].begin(),adj[i].end());
        adj[i].erase(unique(adj[i].begin(),adj[i].end()),adj[i].end());
    }

    for(int i=0;i<n;i++){
        for(int j:adj[i]){
            if(link_delay[i].count(j)==0){
                rng.seed(5000 + min(i,j)*10000 + max(i,j));
                double d = 1.0 + (rng()%9000)/1000.0; // delay 1.0 to 10.0
                link_delay[i][j] = d;
                link_delay[j][i] = d;
            }
        }
    }

    vector<double> dist(n, INF);
    dist[rank] = 0.0;
    for(auto &p: link_delay[rank]) dist[p.first] = p.second;

    bool global_changed = true;
    int rounds = 0;
    while(true){
        rounds++;
        vector<double> sendbuf = dist;
        bool local_changed = false;

        for(int nb: adj[rank]){
            MPI_Send(sendbuf.data(), n, MPI_DOUBLE, nb, 0, MPI_COMM_WORLD);
        }

        for(int nb: adj[rank]){
            vector<double> recv(n);
            MPI_Recv(recv.data(), n, MPI_DOUBLE, nb, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            double w = link_delay[rank][nb];
            for(int k=0;k<n;k++){
                if(recv[k] >= INF/2) continue;
                double cand = w + recv[k];
                if(cand + 1e-9 < dist[k]){
                    dist[k] = cand;
                    local_changed = true;
                }
            }
        }

        int lchg = local_changed?1:0;
        int gchg = 0;
        MPI_Allreduce(&lchg, &gchg, 1, MPI_INT, MPI_SUM, MPI_COMM_WORLD);
        if(gchg==0) break;
        if(rounds > n+5) break;
    }

    vector<double> all_vec(n * n);
    MPI_Allgather(dist.data(), n, MPI_DOUBLE, all_vec.data(), n, MPI_DOUBLE, MPI_COMM_WORLD);

    auto D = [&](int u,int v)->double{ return all_vec[u*n + v]; };

    double Tx = 0.0;
    for(int s=0;s<n;s++){
        if(s==rank) continue;
        double d = D(rank, s);
        if(d >= INF/2) d = 1e9;
        Tx += d;
    }

    double closeness = (n>1) ? ((double)(n-1) / Tx) : 0.0;

    vector<int> next_hop(n, -1);
    for(int s=0;s<n;s++){
        if(s==rank) continue;
        double best = D(rank, s);
        int choice = -1;
        for(int y: adj[rank]){
            double w = link_delay[rank][y];
            double dy_s = D(y, s);
            if(dy_s >= INF/2) continue;
            if(fabs((w + dy_s) - best) < 1e-6){
                if(choice==-1 || y < choice) choice = y;
            }
        }
        next_hop[s] = choice;
    }

    unordered_map<int,int> Ixy, Oxy;
    for(int y: adj[rank]) Ixy[y]=0;
    for(int s=0;s<n;s++){
        if(s==rank) continue;
        int nh = next_hop[s];
        if(nh!=-1) Ixy[nh]++;
    }

    vector<int> localI(n,0);
    for(auto &kv: Ixy) localI[kv.first] = kv.second;

    vector<int> allI(n*n,0);
    MPI_Allgather(localI.data(), n, MPI_INT, allI.data(), n, MPI_INT, MPI_COMM_WORLD);

    for(int y: adj[rank]){
        Oxy[y] = allI[y*n + rank];
    }

    unordered_map<int,bool> phi;
    for(int y: adj[rank]){
        int ixy = Ixy[y];
        int oxy = Oxy[y];
        int delta = oxy - ixy;

        double Ty = 0.0;
        for(int s=0;s<n;s++){
            if(s==y) continue;
            double d = D(y, s);
            if(d >= INF/2) d = 1e9;
            Ty += d;
        }

        bool ok=false;
        if(delta > 0) ok = true;
        else if(delta < 0) ok = false;
        else ok = (Tx < Ty) || (fabs(Tx-Ty)<1e-6 && rank<y);

        phi[y] = ok;
    }

    bool candidacy = true;
    for(int y: adj[rank]) if(!phi[y]) candidacy=false;

    double myC = candidacy ? closeness : -1.0;
    vector<double> allC(n);
    MPI_Allgather(&myC, 1, MPI_DOUBLE, allC.data(), 1, MPI_DOUBLE, MPI_COMM_WORLD);

    double bestC = -1.0;
    int leader = -1;
    for(int u=0;u<n;u++){
        if(allC[u] > bestC){
            bestC = allC[u];
            leader = u;
        }
    }

    int parent = -1;
    if(rank != leader){
        double best = D(rank, leader);
        int choice = -1;
        for(int y: adj[rank]){
            double w = link_delay[rank][y];
            double dy = D(y, leader);
            if(fabs(w + dy - best) < 1e-6){
                if(choice==-1 || y < choice) choice = y;
            }
        }
        parent = choice;
    }

    vector<int> all_parents(n);
    MPI_Allgather(&parent,1,MPI_INT, all_parents.data(),1,MPI_INT,MPI_COMM_WORLD);

    if(rank==0){
        cout<<"=== ZePoP MPI Output ===\nLeader: "<<leader<<"\n";
        for(int i=0;i<n;i++){
            cout<<"Node "<<i<<" -> Parent: "<<all_parents[i]<<"\n";
        }
    }

    MPI_Finalize();
    return 0;
}

EOF

# Compile
mpicxx -std=c++11 -O2 zepop.cpp -o zepop

# Run
mpirun -np 8 ./zepop


SyntaxError: invalid syntax (ipython-input-1494026783.py, line 2)

In [8]:
# Install MPI
apt-get update -y
apt-get install -y mpich

# Add your code
cat << 'EOF' > zepop.cpp

#include <mpi.h>
#include <bits/stdc++.h>
using namespace std;
const double INF = 1e18;

int main(int argc,char**argv){
    MPI_Init(&argc,&argv);
    int rank,size;
    MPI_Comm_rank(MPI_COMM_WORLD,&rank);
    MPI_Comm_size(MPI_COMM_WORLD,&size);
    int n=size;

    std::mt19937_64 rng(12345);
    vector<vector<int>> adj(n);
    vector<unordered_map<int,double>> link_delay(n);

    for(int i=0;i<n;i++){
        int a=(i+1)%n, b=(i-1+n)%n;
        adj[i].push_back(a); adj[i].push_back(b);
    }

    for(int i=0;i<n;i++){
        rng.seed(1000+i);
        int extra = (i*7+13) % n;
        if(extra!=i && find(adj[i].begin(),adj[i].end(),extra)==adj[i].end()){
            adj[i].push_back(extra);
            adj[extra].push_back(i);
        }
    }

    for(int i=0;i<n;i++){
        sort(adj[i].begin(),adj[i].end());
        adj[i].erase(unique(adj[i].begin(),adj[i].end()),adj[i].end());
    }

    for(int i=0;i<n;i++){
        for(int j:adj[i]){
            if(link_delay[i].count(j)==0){
                rng.seed(5000 + min(i,j)*10000 + max(i,j));
                double d = 1.0 + (rng()%9000)/1000.0; // delay 1.0 to 10.0
                link_delay[i][j] = d;
                link_delay[j][i] = d;
            }
        }
    }

    vector<double> dist(n, INF);
    dist[rank] = 0.0;
    for(auto &p: link_delay[rank]) dist[p.first] = p.second;

    bool global_changed = true;
    int rounds = 0;
    while(true){
        rounds++;
        vector<double> sendbuf = dist;
        bool local_changed = false;

        for(int nb: adj[rank]){
            MPI_Send(sendbuf.data(), n, MPI_DOUBLE, nb, 0, MPI_COMM_WORLD);
        }

        for(int nb: adj[rank]){
            vector<double> recv(n);
            MPI_Recv(recv.data(), n, MPI_DOUBLE, nb, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            double w = link_delay[rank][nb];
            for(int k=0;k<n;k++){
                if(recv[k] >= INF/2) continue;
                double cand = w + recv[k];
                if(cand + 1e-9 < dist[k]){
                    dist[k] = cand;
                    local_changed = true;
                }
            }
        }

        int lchg = local_changed?1:0;
        int gchg = 0;
        MPI_Allreduce(&lchg, &gchg, 1, MPI_INT, MPI_SUM, MPI_COMM_WORLD);
        if(gchg==0) break;
        if(rounds > n+5) break;
    }

    vector<double> all_vec(n * n);
    MPI_Allgather(dist.data(), n, MPI_DOUBLE, all_vec.data(), n, MPI_DOUBLE, MPI_COMM_WORLD);

    auto D = [&](int u,int v)->double{ return all_vec[u*n + v]; };

    double Tx = 0.0;
    for(int s=0;s<n;s++){
        if(s==rank) continue;
        double d = D(rank, s);
        if(d >= INF/2) d = 1e9;
        Tx += d;
    }

    double closeness = (n>1) ? ((double)(n-1) / Tx) : 0.0;

    vector<int> next_hop(n, -1);
    for(int s=0;s<n;s++){
        if(s==rank) continue;
        double best = D(rank, s);
        int choice = -1;
        for(int y: adj[rank]){
            double w = link_delay[rank][y];
            double dy_s = D(y, s);
            if(dy_s >= INF/2) continue;
            if(fabs((w + dy_s) - best) < 1e-6){
                if(choice==-1 || y < choice) choice = y;
            }
        }
        next_hop[s] = choice;
    }

    unordered_map<int,int> Ixy, Oxy;
    for(int y: adj[rank]) Ixy[y]=0;
    for(int s=0;s<n;s++){
        if(s==rank) continue;
        int nh = next_hop[s];
        if(nh!=-1) Ixy[nh]++;
    }

    vector<int> localI(n,0);
    for(auto &kv: Ixy) localI[kv.first] = kv.second;

    vector<int> allI(n*n,0);
    MPI_Allgather(localI.data(), n, MPI_INT, allI.data(), n, MPI_INT, MPI_COMM_WORLD);

    for(int y: adj[rank]){
        Oxy[y] = allI[y*n + rank];
    }

    unordered_map<int,bool> phi;
    for(int y: adj[rank]){
        int ixy = Ixy[y];
        int oxy = Oxy[y];
        int delta = oxy - ixy;

        double Ty = 0.0;
        for(int s=0;s<n;s++){
            if(s==y) continue;
            double d = D(y, s);
            if(d >= INF/2) d = 1e9;
            Ty += d;
        }

        bool ok=false;
        if(delta > 0) ok = true;
        else if(delta < 0) ok = false;
        else ok = (Tx < Ty) || (fabs(Tx-Ty)<1e-6 && rank<y);

        phi[y] = ok;
    }

    bool candidacy = true;
    for(int y: adj[rank]) if(!phi[y]) candidacy=false;

    double myC = candidacy ? closeness : -1.0;
    vector<double> allC(n);
    MPI_Allgather(&myC, 1, MPI_DOUBLE, allC.data(), 1, MPI_DOUBLE, MPI_COMM_WORLD);

    double bestC = -1.0;
    int leader = -1;
    for(int u=0;u<n;u++){
        if(allC[u] > bestC){
            bestC = allC[u];
            leader = u;
        }
    }

    int parent = -1;
    if(rank != leader){
        double best = D(rank, leader);
        int choice = -1;
        for(int y: adj[rank]){
            double w = link_delay[rank][y];
            double dy = D(y, leader);
            if(fabs(w + dy - best) < 1e-6){
                if(choice==-1 || y < choice) choice = y;
            }
        }
        parent = choice;
    }

    vector<int> all_parents(n);
    MPI_Allgather(&parent,1,MPI_INT, all_parents.data(),1,MPI_INT,MPI_COMM_WORLD);

    if(rank==0){
        cout<<"=== ZePoP MPI Output ===\nLeader: "<<leader<<"\n";
        for(int i=0;i<n;i++){
            cout<<"Node "<<i<<" -> Parent: "<<all_parents[i]<<"\n";
        }
    }

    MPI_Finalize();
    return 0;
}

EOF

# Compile
mpicxx -std=c++11 -O2 zepop.cpp -o zepop

# Run
mpirun -np 8 ./zepop


SyntaxError: invalid syntax (ipython-input-1494026783.py, line 2)

In [9]:
!apt-get update -y
!apt-get install -y mpich


Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [10]:
%%bash
cat << 'EOF' > zepop.cpp
#include <mpi.h>
#include <bits/stdc++.h>
using namespace std;

const double INF = 1e18;

int main(int argc, char** argv) {

    MPI_Init(&argc, &argv);
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    int n = size;

    std::mt19937_64 rng(12345);
    vector<vector<int>> adj(n);
    vector<unordered_map<int, double>> link_delay(n);

    for (int i = 0; i < n; i++) {
        int a = (i + 1) % n, b = (i - 1 + n) % n;
        adj[i].push_back(a);
        adj[i].push_back(b);
    }

    for (int i = 0; i < n; i++) {
        rng.seed(1000 + i);
        int extra = (i * 7 + 13) % n;
        if (extra != i && find(adj[i].begin(), adj[i].end(), extra) == adj[i].end()) {
            adj[i].push_back(extra);
            adj[extra].push_back(i);
        }
    }

    for (int i = 0; i < n; i++) {
        sort(adj[i].begin(), adj[i].end());
        adj[i].erase(unique(adj[i].begin(), adj[i].end()), adj[i].end());
    }

    for (int i = 0; i < n; i++) {
        for (int j : adj[i]) {
            if (!link_delay[i].count(j)) {
                rng.seed(5000 + min(i, j) * 10000 + max(i, j));
                double d = 1.0 + (rng() % 9000) / 1000.0;
                link_delay[i][j] = d;
                link_delay[j][i] = d;
            }
        }
    }

    vector<double> dist(n, INF);
    dist[rank] = 0.0;
    for (auto &p : link_delay[rank]) dist[p.first] = p.second;

    bool global_changed = true;
    int rounds = 0;
    while (true) {
        rounds++;

        vector<double> sendbuf = dist;
        bool local_changed = false;

        for (int nb : adj[rank]) {
            MPI_Send(sendbuf.data(), n, MPI_DOUBLE, nb, 0, MPI_COMM_WORLD);
        }

        for (int nb : adj[rank]) {
            vector<double> recv(n);
            MPI_Recv(recv.data(), n, MPI_DOUBLE, nb, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
            double w = link_delay[rank][nb];

            for (int k = 0; k < n; k++) {
                if (recv[k] >= INF / 2) continue;
                double cand = w + recv[k];
                if (cand < dist[k]) {
                    dist[k] = cand;
                    local_changed = true;
                }
            }
        }

        int lchg = local_changed ? 1 : 0;
        int gchg = 0;
        MPI_Allreduce(&lchg, &gchg, 1, MPI_INT, MPI_SUM, MPI_COMM_WORLD);

        if (gchg == 0) break;
        if (rounds > n + 5) break;
    }

    vector<double> all_vec(n * n);
    MPI_Allgather(dist.data(), n, MPI_DOUBLE, all_vec.data(), n, MPI_DOUBLE, MPI_COMM_WORLD);

    auto D = [&](int u, int v) { return all_vec[u * n + v]; };

    double Tx = 0.0;
    for (int s = 0; s < n; s++) {
        if (s == rank) continue;
        double d = D(rank, s);
        if (d >= INF / 2) d = 1e9;
        Tx += d;
    }

    double closeness = (n > 1) ? (double)(n - 1) / Tx : 0.0;

    vector<int> next_hop(n, -1);
    for (int s = 0; s < n; s++) {
        if (s == rank) continue;
        double best = D(rank, s);
        int choice = -1;

        for (int y : adj[rank]) {
            double w = link_delay[rank][y];
            double dy = D(y, s);
            if (fabs((w + dy) - best) < 1e-6) {
                if (choice == -1 || y < choice) choice = y;
            }
        }
        next_hop[s] = choice;
    }

    unordered_map<int, int> Ixy, Oxy;
    for (int y : adj[rank]) Ixy[y] = 0;

    for (int s = 0; s < n; s++) {
        if (s == rank) continue;
        int nh = next_hop[s];
        if (nh != -1) Ixy[nh]++;
    }

    vector<int> localI(n, 0);
    for (auto &kv : Ixy) localI[kv.first] = kv.second;

    vector<int> allI(n * n, 0);
    MPI_Allgather(localI.data(), n, MPI_INT, allI.data(), n, MPI_INT, MPI_COMM_WORLD);

    for (int y : adj[rank]) {
        Oxy[y] = allI[y * n + rank];
    }

    unordered_map<int, bool> phi;

    for (int y : adj[rank]) {

        int ixy = Ixy[y], oxy = Oxy[y];
        int delta = oxy - ixy;

        double Ty = 0.0;
        for (int s = 0; s < n; s++) {
            if (s == y) continue;
            double d = D(y, s);
            if (d >= INF / 2) d = 1e9;
            Ty += d;
        }

        bool ok = false;
        if (delta > 0) ok = true;
        else if (delta < 0) ok = false;
        else ok = (Tx < Ty) || (fabs(Tx - Ty) < 1e-6 && rank < y);

        phi[y] = ok;
    }

    bool candidacy = true;
    for (int y : adj[rank]) if (!phi[y]) candidacy = false;

    double myC = candidacy ? closeness : -1.0;
    vector<double> allC(n);
    MPI_Allgather(&myC, 1, MPI_DOUBLE, allC.data(), 1, MPI_DOUBLE, MPI_COMM_WORLD);

    double bestC = -1.0;
    int leader = -1;
    for (int u = 0; u < n; u++) {
        if (allC[u] > bestC) {
            bestC = allC[u];
            leader = u;
        }
    }

    int parent = -1;

    if (rank != leader) {
        double best = D(rank, leader);
        int choice = -1;

        for (int y : adj[rank]) {
            double w = link_delay[rank][y];
            double dy = D(y, leader);
            if (fabs((w + dy) - best) < 1e-6) {
                if (choice == -1 || y < choice) choice = y;
            }
        }
        parent = choice;
    }

    vector<int> all_parents(n);
    MPI_Allgather(&parent, 1, MPI_INT, all_parents.data(), 1, MPI_INT, MPI_COMM_WORLD);

    if (rank == 0) {
        cout << "=== ZePoP MPI Output ===\n";
        cout << "Leader: " << leader << "\n";
        for (int i = 0; i < n; i++) {
            cout << "Node " << i << " -> Parent: " << all_parents[i] << "\n";
        }
    }

    MPI_Finalize();
    return 0;
}

EOF


In [11]:
!mpicxx -std=c++11 -O2 zepop.cpp -o zepop


In [12]:
!mpirun -np 8 ./zepop


--------------------------------------------------------------------------
mpirun has detected an attempt to run as root.

Running as root is *strongly* discouraged as any mistake (e.g., in
defining TMPDIR) or bug can result in catastrophic damage to the OS
file system, leaving your system in an unusable state.

We strongly suggest that you run mpirun as a non-root user.

You can override this protection by adding the --allow-run-as-root option
to the cmd line or by setting two environment variables in the following way:
the variable OMPI_ALLOW_RUN_AS_ROOT=1 to indicate the desire to override this
protection, and OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 to confirm the choice and
add one more layer of certainty that you want to do so.
We reiterate our advice against doing so - please proceed at your own risk.
--------------------------------------------------------------------------


In [13]:
!mpirun --allow-run-as-root --oversubscribe -np 8 ./zepop


=== ZePoP MPI Output ===
Leader: 4
Node 0 -> Parent: 5
Node 1 -> Parent: 4
Node 2 -> Parent: 1
Node 3 -> Parent: 4
Node 4 -> Parent: -1
Node 5 -> Parent: 4
Node 6 -> Parent: 5
Node 7 -> Parent: 0
